In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [2]:
sqlContext.sql("use dsc60230_gcct_tz_db")

DataFrame[]

In [5]:
import xlrd
old=pd.ExcelFile('/s/latluri1/DES/GCCT_schema_Jan19.xlsx').parse("sheet1")
new=pd.ExcelFile('GCCT_schema_Mar2019.xlsx').parse("sheet1")

In [2]:
new=pd.read_csv('/s/latluri1/GCCT_DES_DSC_monitoring/PPROD/archive/GCCT_schema_DSC_QA_Aug_19_2019_LZ.csv')
old=pd.read_csv('/s/latluri1/GCCT_DES_DSC_monitoring/PPROD/archive/GCCT_schema_DSC_QA_June_11_2019_LZ.csv')

In [9]:
final=old.merge(new,how='outer',on=['table','col_name'],suffixes=('_old','_new') )
final

,col_name,data_type_old,table,data_type_new
0,account_identity,int,account,int
1,account_rowversion,string,account,string
2,id,string,account,string
3,sinkcreatedon,timestamp,account,timestamp
4,sinkmodifiedon,timestamp,account,timestamp
5,statecode,int,account,int
6,statuscode,int,account,int
7,customertypecode,int,account,int
8,address1_freighttermscode,int,account,int
9,address1_addresstypecode,int,account,int


# ADDED

In [12]:
final[final.data_type_old.isnull()]

,col_name,data_type_old,table,data_type_new
12341,gcct_dealerportalrole,NaN,contact,string
12342,gcct_counterofferedbuybacktype,NaN,gcct_arbitrationclaimprocessing,string
12343,gcct_d2ddealerswithstock,NaN,gcct_partssmeassistancerequest,int
12344,gcct_isassigntoteam,NaN,incident,boolean


In [1]:
#gcct_counterofferedbuybacktype is int in infor schema



# Removed

In [13]:
final[final.data_type_new.isnull()]

,col_name,data_type_old,table,data_type_new
2494,gcct_startassignmentbyid,string,gcct_caseassignment,NaN
2510,gcct_resolvebyid,string,gcct_caseassignment,NaN
2583,gcct_datenow,timestamp,gcct_caseassignment,NaN
9212,gcct_vehicleonwercustomerid,string,gcct_vehicleowner,NaN


# Dtype changed

In [18]:
final.loc[(final.data_type_new!=final.data_type_old) & (np.logical_not(final.data_type_new.isnull())) & (np.logical_not(final.data_type_old.isnull()))]

,col_name,data_type_old,table,data_type_new
743,effort,float,calendarrule,"decimal(20,10)"
744,extentcode,float,calendarrule,"decimal(20,10)"
5877,gcct_vehicleoffroad,boolean,gcct_partssmeassistancerequest,string
5882,gcct_customerinrental,boolean,gcct_partssmeassistancerequest,string


In [ ]:
#gcct_vehicleoffroad is int
#gcct_customerinrental in int